Importing dataset

In [ ]:
import os
import pandas as pd
import numpy as np
import spacy
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
from collections import Counter
import pickle

In [ ]:
# Method 1: uploading from local disk

from google.colab import files
files.upload()

{}

In [ ]:
# An alternative to access dataset directly from Google Drive, taking few seconds to load the entire dataset

# !pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1N_vtjPrOTmD3HOExYRA-sFSD_ChKaMmP"})
downloaded.GetContentFile('rcd_001.csv')

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

In [ ]:
# Read the dataset

df_tweets = pd.read_csv('rcd_001.csv', index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Checking the dataframe

In [ ]:
df_tweets.head()

ID  ...                               Further_Cleaned_Text
0  6.215471e+07  ...  focus texans period    worry hillary hell lot ...
1  1.114481e+08  ...  suddenly discover 1st amendment like forget ea...
2  1.188767e+08  ...  evidence drink vodka actually free novel coron...
3  8.294621e+17  ...  want life approach like normal anytime soon ne...
4  1.240522e+18  ...  subhuman think intelligent rolling on the floo...

[5 rows x 4 columns]

In [ ]:
df_tweets.dropna(inplace=True) # Null values would cause error in later codes

Initial Analysis

In [ ]:
#Build a word count line graph to explore density of the terms#
print('Pulling out indivdual words.')
all_words = []
for line in df_tweets['Further_Cleaned_Text']:
    words = line.split()
    for word in words:
        all_words.append(word.lower())
print('Words extracted.')
print(all_words[:25])

word_count = Counter(all_words).most_common(1000) # can manipulate the total number of words counted (e.g. 10,000; 30,000, etc.)
word_count_x = []
word_count_y = []
for word, count in word_count:
    word_count_x.append(word)
    word_count_y.append(count)


plt.figure(figsize=(22,10))
plt.plot(word_count_x[850:], word_count_y[850:], linestyle='-', linewidth=1) # play around with the values within x and y to slice the data differently
plt.ylabel("Count")
plt.xlabel("Word")
plt.xticks(fontsize='small', rotation=90)
plt.title('Plot of words frequency in corpus')
plt.show()

Modeling: TF-IDF

In [ ]:
# TF-IDF

tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=['fuck', 'shit', 'ass'])

dtm = tfidf.fit_transform(df_tweets['Further_Cleaned_Text'])

Modeling: K-means Clustering

In [ ]:
# Identify the optimal K with Silhouette Analysis

from sklearn.metrics import silhouette_samples, silhouette_score

range_n_clusters = list(range(2,10))
sil_scores = []

for n_clusters in range_n_clusters:
    
    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 3 for reproducibility.
    clusterer = MiniBatchKMeans(n_clusters=n_clusters, batch_size=200, random_state=3)
    cluster_labels = clusterer.fit_predict(dtm)

    # The silhouette_score gives the average value for all the samples.
    silhouette_avg = silhouette_score(dtm, cluster_labels)
    sil_scores.append(silhouette_avg)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

print("The best K is", " ", sil_scores.index(max(sil_scores))+1, "with a silhouette score of", " ", max(sil_scores))
    

NameError: ignored

In [ ]:
# Assign each record to a cluster

# According to the plot, select the optimal K
K = sil_scores.index(max(sil_scores))+1

# Instantiate the model
km = KMeans(n_clusters=K, random_state=42)

# Fit the model with data
km.fit(dtm)

# Predict the cluster of each section_title
df_tweets['clusters'] = km.predict(dtm)

In [ ]:
# Print 10 documents out of each cluster

def review_clusters(df, n_clusters):
    for cl_num in range(n_clusters):
        print(cl_num)
        print(df_tweets[df.clusters == cl_num]['Further_Cleaned_Text'].values[0:10])
        print()

review_clusters(df_tweets, n_clusters=K)

In [ ]:
# SVD

from sklearn.decomposition import TruncatedSVD # to work with sparse matrices
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt

sk_tsvd = TruncatedSVD(n_components=2)

y_tsvd = sk_tsvd.fit_transform(titles_vect)

km = KMeans(n_clusters=K, random_state=42)

# fit to TruncSVD values
km.fit(y_tsvd)

# predict clusters
svd_clusters = km.predict(y_tsvd)

# plot results
plt.scatter(y_tsvd[:, 0], y_tsvd[:, 1], c=svd_clusters, s=50, cmap='magma')
plt.show()

Modeling: Sentiment Analysis



In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# create a function to pass our sentences
def sentiment_analyzer_scores(sentence):
    score = analyzer.polarity_scores(sentence)
    #print("{:-<60} {}".format(sentence, str(score)))
    return score

In [ ]:
l=[str(x) for x in df_tweets['Further_Cleaned_Text']]

In [ ]:
# let's apply to our Amazon review dataset and see how well the data was labeled
# we will specifically pull out the 'compound score
df_tweets['score'] = [sentiment_analyzer_scores(review)['compound'] for review in l]

df_tweets.head(10)

In [ ]:
df_tweets['V_Sentiment'] = df_tweets['score'].apply(lambda c: 'pos' if c >=0.05 else ('neu' if c > -0.05 else 'neg'))

df_tweets.head(50)

Sentiment Analysis (state wise) to see which states’ tweets have what percentage of +ve and -ve tweets


In [ ]:
SentimentByState=pd.DataFrame(df_tweets.groupby(by=['Location','V_Sentiment'])['V_Sentiment'].count())

In [ ]:
SentimentByState

In [ ]:
state_pcts = SentimentByState.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [ ]:
state_pcts

In [ ]:
state_pcts.to_csv('rcd_state_sentimentPercent.csv', index=False) # this will take a while

In [ ]:
state_pcts.to_csv(r'C:\Users\victo\Desktop\filename.csv', index=False)

In [ ]:
SentimentByState.to_csv('rcd_state_sentimentCount.csv', index=False) # this will take a while

In [ ]:
from google.colab import files
files.download('rcd_state_sentimentCount.csv')